# Sequence Utilities

> Utilities for working with DNA/RNA sequences and barcode detection

## Introduction
This notebook contains utility functions for sequence manipulation, barcode detection, and quality assessment in the BarcodeSeqKit library. These utilities are format-agnostic and can be used with both BAM and FASTQ data.

In [ ]:
#| default_exp sequence_utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| export
import re
import regex  # For fuzzy matching
from typing import List, Dict, Tuple, Optional, Iterator, Set, Any, Union
from Bio.Seq import Seq
from BarcodeSeqKit.core import BarcodeConfig, BarcodeMatch, OrientationType, BarcodeLocationType

## Basic Sequence Operations
First, let's define basic sequence manipulation functions.

In [ ]:
#| export
def reverse_complement(sequence: str) -> str:
    """Return the reverse complement of a DNA sequence.
    
    Args:
        sequence: DNA sequence
        
    Returns:
        Reverse complement of the sequence
    """
    return str(Seq(sequence).reverse_complement())

In [ ]:
#| export
def hamming_distance(seq1: str, seq2: str) -> int:
    """Calculate the Hamming distance between two sequences.
    
    The sequences must be of the same length.
    
    Args:
        seq1: First sequence
        seq2: Second sequence
        
    Returns:
        Hamming distance (number of positions where the sequences differ)
        
    Raises:
        ValueError: If sequences have different lengths
    """
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must have the same length")
    
    return sum(a != b for a, b in zip(seq1, seq2))

## Barcode Detection
Now let's implement functions for detecting barcodes in sequences.

In [ ]:
#| export
def find_barcode_matches(
    sequence: str, 
    barcodes: List[BarcodeConfig],
    max_mismatches: int = 0
) -> List[BarcodeMatch]:
    """Find all barcode matches in a sequence.
    
    Args:
        sequence: The DNA/RNA sequence to search in
        barcodes: List of barcode configurations to search for
        max_mismatches: Maximum number of mismatches to allow
        
    Returns:
        List of BarcodeMatch objects representing the matches found
    """
    matches = []
    
    # Convert sequence to uppercase
    sequence = sequence.upper()
    
    for barcode in barcodes:
        # Search for forward sequence
        if max_mismatches == 0:
            # Exact matching with re
            forward_positions = [m.start() for m in re.finditer(barcode.sequence, sequence)]
            for pos in forward_positions:
                matches.append(BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.FORWARD,
                    position=pos,
                    sequence=sequence[pos:pos+len(barcode.sequence)]
                ))
            
            # Search for reverse complement sequence
            rc_seq = barcode.reverse_complement
            rc_positions = [m.start() for m in re.finditer(rc_seq, sequence)]
            for pos in rc_positions:
                matches.append(BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.REVERSE_COMPLEMENT,
                    position=pos,
                    sequence=sequence[pos:pos+len(rc_seq)]
                ))
        else:
            # Fuzzy matching with regex module
            # Forward orientation
            pattern = f"({barcode.sequence}){{e<={max_mismatches}}}"
            r = regex.compile(pattern)
            for match in r.finditer(sequence):
                matches.append(BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.FORWARD,
                    position=match.start(),
                    sequence=match.group()
                ))
            
            # Reverse complement orientation
            rc_pattern = f"({barcode.reverse_complement}){{e<={max_mismatches}}}"
            rc_r = regex.compile(rc_pattern)
            for match in rc_r.finditer(sequence):
                matches.append(BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.REVERSE_COMPLEMENT,
                    position=match.start(),
                    sequence=match.group()
                ))
    
    return matches

In [ ]:
#| export
def find_best_barcode_match(
    sequence: str, 
    barcodes: List[BarcodeConfig],
    max_mismatches: int = 1
) -> Optional[BarcodeMatch]:
    """Find the best matching barcode in a sequence.
    
    Args:
        sequence: The sequence to search in
        barcodes: List of barcode configurations to search for
        max_mismatches: Maximum number of mismatches to allow
        
    Returns:
        The best matching BarcodeMatch or None if no match found
    """
    best_match = None
    min_mismatches = max_mismatches + 1
    sequence = sequence.upper()
    
    for barcode in barcodes:
        # Check forward orientation with regex
        pattern = f"({barcode.sequence}){{e<={max_mismatches}}}"
        r = regex.compile(pattern)
        for match in r.finditer(sequence):
            # Get the fuzzy counts (substitutions, insertions, deletions)
            subs, ins, dels = match.fuzzy_counts
            total_mismatches = subs + ins + dels
            
            if total_mismatches < min_mismatches:
                min_mismatches = total_mismatches
                best_match = BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.FORWARD,
                    position=match.start(),
                    sequence=match.group()
                )
        
        # Check reverse complement orientation with regex
        rc_pattern = f"({barcode.reverse_complement}){{e<={max_mismatches}}}"
        rc_r = regex.compile(rc_pattern)
        for match in rc_r.finditer(sequence):
            # Get the fuzzy counts
            subs, ins, dels = match.fuzzy_counts
            total_mismatches = subs + ins + dels
            
            if total_mismatches < min_mismatches:
                min_mismatches = total_mismatches
                best_match = BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.REVERSE_COMPLEMENT,
                    position=match.start(),
                    sequence=match.group()
                )
    
    return best_match

In [ ]:
#| export
def classify_read_by_first_match(
    sequence: str, 
    barcodes: List[BarcodeConfig],
    max_mismatches: int = 0
) -> Tuple[Optional[BarcodeMatch], str]:
    """Classify a read based on the first barcode match found.
    
    This is an optimized version that stops after finding the first match,
    without evaluating all possible matches in the sequence.
    
    Args:
        sequence: The sequence to classify
        barcodes: List of barcode configurations to search for
        max_mismatches: Maximum number of mismatches to allow
        
    Returns:
        Tuple of (match, category)
        Category is one of: "barcode5_orientFR", "barcode5_orientRC", etc.
        or "noBarcode" if no match found
    """
    sequence = sequence.upper()
    
    # Determine if we're in single barcode mode or multiple barcode mode
    single_barcode_mode = len(barcodes) == 1 or all(b.location.value == "UNK" for b in barcodes)
    
    # Check each barcode in order
    for barcode in barcodes:
        # Check forward orientation first
        if max_mismatches == 0:
            # Exact match with re
            match = re.search(barcode.sequence, sequence)
            if match:
                barcode_match = BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.FORWARD,
                    position=match.start(),
                    sequence=match.group()
                )
                
                if single_barcode_mode:
                    return barcode_match, "barcode_orientFR"
                else:
                    location = barcode.location.value if barcode.location.value in ["5", "3"] else ""
                    return barcode_match, f"barcode{location}_orient{OrientationType.FORWARD.value}"
        else:
            # Fuzzy match with regex
            pattern = f"({barcode.sequence}){{e<={max_mismatches}}}"
            r = regex.compile(pattern)
            match = r.search(sequence)
            if match:
                barcode_match = BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.FORWARD,
                    position=match.start(),
                    sequence=match.group()
                )
                
                if single_barcode_mode:
                    return barcode_match, "barcode_orientFR"
                else:
                    location = barcode.location.value if barcode.location.value in ["5", "3"] else ""
                    return barcode_match, f"barcode{location}_orient{OrientationType.FORWARD.value}"
        
        # Check reverse complement orientation
        rc_seq = barcode.reverse_complement
        if max_mismatches == 0:
            # Exact match with re
            match = re.search(rc_seq, sequence)
            if match:
                barcode_match = BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.REVERSE_COMPLEMENT,
                    position=match.start(),
                    sequence=match.group()
                )
                
                if single_barcode_mode:
                    return barcode_match, "barcode_orientRC"
                else:
                    location = barcode.location.value if barcode.location.value in ["5", "3"] else ""
                    return barcode_match, f"barcode{location}_orient{OrientationType.REVERSE_COMPLEMENT.value}"
        else:
            # Fuzzy match with regex
            pattern = f"({rc_seq}){{e<={max_mismatches}}}"
            r = regex.compile(pattern)
            match = r.search(sequence)
            if match:
                barcode_match = BarcodeMatch(
                    barcode=barcode,
                    orientation=OrientationType.REVERSE_COMPLEMENT,
                    position=match.start(),
                    sequence=match.group()
                )
                
                if single_barcode_mode:
                    return barcode_match, "barcode_orientRC"
                else:
                    location = barcode.location.value if barcode.location.value in ["5", "3"] else ""
                    return barcode_match, f"barcode{location}_orient{OrientationType.REVERSE_COMPLEMENT.value}"
    
    # No match found
    return None, "noBarcode"

## Category and Output File Handling

In [ ]:
#| export
def get_output_category(
    match: Optional[BarcodeMatch], 
    single_barcode_mode: bool = False
) -> str:
    """Get the output category for a barcode match.
    
    Args:
        match: Barcode match or None
        single_barcode_mode: Whether we're in single barcode mode
        
    Returns:
        Category string for output file naming
    """
    if match is None:
        return "noBarcode"
    
    if single_barcode_mode:
        return f"barcode_orient{match.orientation.value}"
    else:
        location = match.barcode.location.value
        return f"barcode{location}_orient{match.orientation.value}"

## Example Usage
Let's demonstrate how to use these utilities.

In [ ]:
# Create test barcode configurations
from BarcodeSeqKit.core import BarcodeConfig, BarcodeLocationType

barcode_5prime = BarcodeConfig(
    sequence="TCGCGAGGC",
    location=BarcodeLocationType.FIVE_PRIME,
    name="5",
    description="5' barcode for test"
)

barcode_3prime = BarcodeConfig(
    sequence="GGCCGGCCGG",
    location=BarcodeLocationType.THREE_PRIME,
    name="3",
    description="3' barcode for test"
)

# Example sequence with barcodes
sequence = "AAAAAATCGCGAGGCAAAAAAAGGCCGGCCGGAAAAAA"
print(f"Test sequence: {sequence}")

# Find all barcode matches
matches = find_barcode_matches(sequence, [barcode_5prime, barcode_3prime])
print(f"Found {len(matches)} matches:")
for match in matches:
    print(f"  {match}")
    print(f"    Barcode: {match.barcode.name}")
    print(f"    Orientation: {match.orientation.value}")
    print(f"    Position: {match.position}")
    print(f"    Sequence: {match.sequence}")

# Classify a read using first match
print("\nClassifying reads:")
for test_seq in [sequence, 
                "AAAAAAGCCTCGCGAAAAAAA",  # 5' barcode with mismatch
                "AAAAAAGGCCGGCCTGAAAAAA"]: # 3' barcode with mismatch
    match, category = classify_read_by_first_match(
        sequence=test_seq,
        barcodes=[barcode_5prime, barcode_3prime],
        max_mismatches=1
    )
    print(f"Sequence: {test_seq}")
    print(f"  Match: {match}")
    print(f"  Category: {category}")

Test sequence: AAAAAATCGCGAGGCAAAAAAAGGCCGGCCGGAAAAAA
Found 2 matches:
  5 (FR) at position 6
    Barcode: 5
    Orientation: FR
    Position: 6
    Sequence: TCGCGAGGC
  3 (FR) at position 22
    Barcode: 3
    Orientation: FR
    Position: 22
    Sequence: GGCCGGCCGG

Classifying reads:
Sequence: AAAAAATCGCGAGGCAAAAAAAGGCCGGCCGGAAAAAA
  Match: 5 (FR) at position 5
  Category: barcode5_orientFR
Sequence: AAAAAAGCCTCGCGAAAAAAA
  Match: 5 (RC) at position 5
  Category: barcode5_orientRC
Sequence: AAAAAAGGCCGGCCTGAAAAAA
  Match: 3 (FR) at position 6
  Category: barcode3_orientFR


In [ ]:
# Test with real data 
import os
import pysam
from tqdm.auto import tqdm
from BarcodeSeqKit.core import ExtractionStatistics

# Path to the test file (adjust if needed)
bam_file = "../tests/test.bam"

if os.path.exists(bam_file):
    print(f"Testing with {bam_file}")
    stats = ExtractionStatistics()
    
    # Define barcodes to search for
    example_barcodes = [
        BarcodeConfig(
            sequence="TAACTGAGGCCGGC",  # Example barcode to search for
            location=BarcodeLocationType.THREE_PRIME,
            name="3prime",
            description="3' barcode from test data"
        ),
        BarcodeConfig(
            sequence="CTGACTCCTTAAGGGCC",  # Example barcode to search for
            location=BarcodeLocationType.FIVE_PRIME,
            name="5prime",
            description="5' barcode from test data"
        )
    ]
    
    # Count matches
    with pysam.AlignmentFile(bam_file, "rb") as bam:
        for read in tqdm(bam):
            stats.total_reads += 1
            sequence = read.query_sequence
            if sequence:
                match, category = classify_read_by_first_match(
                    sequence=sequence,
                    barcodes=example_barcodes,
                    max_mismatches=0
                )
                if match:
                    stats.update_barcode_match(match, category)
    
    # Print statistics
    print("\nBarcode detection statistics:")
    print(f"Total reads: {stats.total_reads}")
    print(f"Total matches: {stats.total_barcode_matches}")
    
    for barcode_name, count in stats.matches_by_barcode.items():
        print(f"  {barcode_name}: {count} matches")
    
    for orientation, count in stats.matches_by_orientation.items():
        print(f"  Orientation {orientation}: {count} matches")
    
    for category, count in stats.matches_by_category.items():
        print(f"  Category {category}: {count} matches")
else:
    print(f"Test file not found: {bam_file}")

Testing with ../tests/test.bam


0it [00:00, ?it/s]


Barcode detection statistics:
Total reads: 498
Total matches: 18
  5prime: 10 matches
  3prime: 8 matches
  Orientation FR: 10 matches
  Orientation RC: 8 matches
  Category barcode5_orientFR: 7 matches
  Category barcode3_orientFR: 3 matches
  Category barcode3_orientRC: 5 matches
  Category barcode5_orientRC: 3 matches


## Conclusion
This notebook provides utility functions for sequence manipulation, barcode detection, and classification in BarcodeSeqKit. These functions are used by both the BAM and FASTQ processing modules to identify and categorize barcoded reads.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()